In [1]:
import logging
log_level = logging.INFO
# log_level = logging.DEBUG
log_level = 15
logging.basicConfig(level=log_level)
logger = logging.getLogger(__name__)
logger.setLevel(log_level)


In [18]:
import os
import importlib

from openai import OpenAI

from regulations_rag.file_tools import load_parquet_data, save_parquet_data


import data_tools.index_tools
importlib.reload(data_tools.index_tools)
from data_tools.index_tools import update_text_in_index, add_to_index, remove_from_index

openai_client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"),)
key = os.getenv('encryption_key_gdpr')

#data  = CEMADData(key)

In [3]:
file = "./inputs/index/consent.parquet"

doc_index = load_parquet_data(file, key)

embedding_model =  "text-embedding-3-large"
embedding_dimensions = 1024


In [14]:
tmp = doc_index[doc_index["section_reference"] == "3.1.1"]
for index, row in tmp.iterrows():
    print("--" + row['text'])

--Freely given consent requires a genuine choice without pressure, intimidation, or significant negative consequences. Public authorities usually cannot rely on consent due to power imbalances, but there are exceptions, such as voluntary subscription to updates on road works.

In employment, the employer-employee relationship creates a dependency that generally undermines the ability to give free consent. Employees are unlikely to freely consent to actions like workplace monitoring due to fear of repercussions. Employers can rely on consent only in exceptional cases where no adverse consequences arise from refusal.

Imbalances of power are not limited to public authorities or employers and can also appear in other contexts. Valid consent requires the absence of compulsion, pressure, or an inability to exercise free will.
--When is there an imbalance of power in data processing?
--Can public authorities rely on consent for processing data?
--Can employers use consent as a basis for data

In [12]:
tmp

,section_reference,text,source,embedding,document
3,3.1.1,Freely given consent requires a genuine choice...,summary,"[-0.03908756002783775, -0.04293598607182503, -...",Consent
30,3.1.1,When is there an imbalance of power in data pr...,question,"[-0.024126196280121803, -0.010877548716962337,...",Consent
31,3.1.1,Can public authorities rely on consent for pro...,question,"[-0.04015662521123886, -0.0452830046415329, -0...",Consent
32,3.1.1,Can employers use consent as a basis for data ...,question,"[-0.027924824506044388, -0.03840619698166847, ...",Consent
33,3.1.1,What constitutes a real choice for consent?,question,"[0.02379072643816471, -0.04665195569396019, -0...",Consent
34,3.1.1,How can employees give free consent to data pr...,question,"[-0.03558824583888054, -0.04410198703408241, -...",Consent


## Add an entry

In [13]:
document = 'Consent'
text_to_add = "Can I covertly monitor my employees without breaching GDPR?"
section_reference = "3.1.1"
source = "question"
doc_index = add_to_index(openai_client = openai_client, index_df = doc_index, text = text_to_add, section_reference = section_reference, source = source, embedding_model = embedding_model, embedding_dimensions= embedding_dimensions, document=document)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [20]:
# SAVE IT
save_parquet_data(doc_index, file, key)

## Change an entry

In [19]:
text_to_change = "Can I covertly monitor my employees without breaching GDPR?"
changed_text = "Am I able to use the messages and calls made by an employee on a business device in a disciplinary without consent from the employee or the recipient of the messages or calls?"

doc_index = update_text_in_index(openai_client = openai_client, index_df = doc_index , text_to_change=text_to_change, changed_text=changed_text, embedding_model=embedding_model, embedding_dimensions=embedding_dimensions)

# SAVE IT
# save_parquet_data(doc_index, file, key)

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
